In [ ]:
import roboticstoolbox as rtb
from spatialmath import SE3
import numpy as np
import imageio

# Definition of the Lego robot arm structure (links and joints). This is a right arm:

studs = 0.008  # 8 mm between Lego studs

# Define joint limits in degrees and convert to radians
joint_limits_deg = [
    [-10, 45],    # Joint 1: +45 to -10 degrees
    [0, 35],      # Joint 2: +35 to 0 degrees  
    [-120, 35],   # Joint 3: +35 to -120 degrees
    [-20, 13],    # Joint 4: +13 to -20 degrees
    [-120, 120],  # Joint 5: +120 to -120 degrees
    [-110, 110]   # Joint 6: +110 to -110 degrees
]
joint_limits_rad = np.deg2rad(joint_limits_deg)

E1 = rtb.ET.tz(0.377)                                 # 37.7 cm up
E2 = rtb.ET.Ry(flip=True, qlim=joint_limits_rad[0])   # Shoulder rotation forward/back                 (positive angle is forwards)
E3 = rtb.ET.ty(-9.5*studs)                            # 9.5 studs out to next shoulder joint
E4 = rtb.ET.Rx(flip=True, qlim=joint_limits_rad[1])   # Shoulder rotation out/up / in/down to the side (positive angle is out/up)
E5 = rtb.ET.tz(-20.5*studs)                           # 20.5 studs down to overarm rotation joint
E6 = rtb.ET.Rz(qlim=joint_limits_rad[2])              # Overarm rotation                               (positive angle is counter clockwise in towards the body)
E7 = rtb.ET.tz(-12.5*studs)                           # 12.5 studs down to elbow
E8 = rtb.ET.Ry(flip=True, qlim=joint_limits_rad[3])   # Elbow rotation                                 (positive angle is flexing the elbow upwards)
E9 = rtb.ET.tx(10.5*studs)                            # 10.5 studs out to underarm rotation joint
E10 = rtb.ET.Rx(qlim=joint_limits_rad[4])             # Underarm rotation                              (positive angle is rotating the underarm clockwise)
E11 = rtb.ET.tx(23.5*studs)                           # 23.5 studs out to wrist rotation joint
E12 = rtb.ET.Rz(qlim=joint_limits_rad[5])             # Wrist rotation up/down                         (positive angle is rotating the wrist downwards/inwards)
E13 = rtb.ET.tx(7*studs)                              # 7 studs out to gripper (TODO: Fix actual length))

robot_arm = E1 * E2 * E3 * E4 * E5 * E6 * E7 * E8 * E9 * E10 * E11 * E12 * E13
print(robot_arm) # View the ETS (Elementary Transformation Sequence, where ET is a translation or rotation).
print(f"The robot has {robot_arm.n} joints")
print(f"The robot has {robot_arm.m} Elementary Transformations (ETs)")

neutral_pose_with_bent_elbow_q = np.array([0, 0, 0, 0, 0, 0])
print(f"\nneutral_pose_with_bent_elbow: \n{robot_arm.fkine(neutral_pose_with_bent_elbow_q)}\n\n") # Calculate the forward kinematics of the neutral pose with a bent elbow, and print it.

#elbow_extended_q = np.deg2rad(np.array([0, 0, 0, 90, 0, 0]))
#print(f"elbow_extended: \n{robot_arm.fkine(elbow_extended_q)}")


# Define a list of deltas to test different positions and orientations
# eacl delta is defined by delta_x (forward), delta_y (left) and delta_z (up)
deltas = [#OK[0.00, -0.30, 0.30],
          #OK[0.00, -0.20, 0.20],
          [-0.05,  0.00, 0.00],
          [-0.05, -0.20, 0.00],
          [-0.05, -0.20, 0.10],
          [ 0.05, -0.20, 0.10],
          [ 0.05, -0.00, 0.10],
          [ 0.05, -0.00, 0.00],
          [ 0.00,  0.00, 0.00],
          ]

start_pose = neutral_pose_with_bent_elbow_q
for index, delta in enumerate(deltas):
    delta_x, delta_y, delta_z = delta
    # Choose a wanted pose SE(3) defined in terms of position and orientation 
    # (end-effector z-axis down (A=-Z) and finger orientation parallel to y-axis (O=+Y)).
    wanted_pose = SE3.Trans(0.328+delta_x, -0.076+delta_y, 0.113+delta_z) * SE3.OA([0, 1, 0], [0, 0, 1])
    #wanted_pose = SE3.Trans(0.328, -0.076, 0.113) * SE3.OA([0, 1, 0], [0, 0, 1]) # Same as starting pose, to test sanity of IK solution and model.
    #wanted_pose = SE3.Trans(0.328, -0.076, 0.113) * SE3.OA([1, 1, 0], [0, 0, 1]) # Rotates the end-effector 45 degrees around the z-axis

    mask_priority = np.array([2, 2, 2, 0, 1, 1])  # We want to prioritize the position over the orientation in the IK solution, 
                                                # and for orientation we don't prioritize rotation around the X axis (should be along the gripper "fingers" so not too important).

    # Solve IK (inverse kinematics):
    joint_angles_solution = robot_arm.ik_LM(Tep=wanted_pose, q0=start_pose, mask=mask_priority, joint_limits=True, ilimit=300, slimit=1000)

    solution_q, reason, iterations, num_searches, residual = joint_angles_solution
    print(f"Inverse kinematics solution #{index}: {'SUCCESS' if reason else 'FAILURE'}")
    #print(f"Termination status: {reason} (1=success, 0=failure)")
    #print(f"Solution found in {iterations} iterations")
    #print(f"Number of searches performed: {num_searches}")
    #print(f"Final residual error: {residual:.6f}")
    print(f"Joint angles suggested (degrees): {np.round(np.rad2deg(solution_q))}")

    # Check if solution respects joint limits
    q_deg = np.rad2deg(solution_q)
    within_limits = True
    for i, (q_val, limits) in enumerate(zip(q_deg, joint_limits_deg)):
        if q_val < limits[0] or q_val > limits[1]:
            print(f"WARNING: Joint {i+1} angle {q_val:.1f}° exceeds limits [{limits[0]}, {limits[1]}]")
            within_limits = False
    if within_limits:
        #print("All joint angles are within specified limits.")
        pass

    #print(f"Wanted pose: \n{wanted_pose}")
    #print(f"Solution: \n{robot_arm.fkine(solution_q)}")

    # We can animate a path from the ready pose (qr configuration) to the new wanted position and orientation
    qt = rtb.jtraj(start_pose, solution_q, 50)
    robot_arm.plot(qt.q, backend='pyplot', movie='robot_arm_vid_' + str(index) + '.gif')

    start_pose = solution_q # Update the start pose for the next iteration


# Combine all created gifs into one gif file:
print("Combining all animated GIFs into one...")
number_of_deltas = len(deltas)
all_frames = []
for i in range(number_of_deltas):
    # Read all frames from each animated GIF
    gif_frames = imageio.mimread(f'robot_arm_vid_{i}.gif')
    all_frames.extend(gif_frames)  # Add all frames to the combined list
imageio.mimsave('robot_arm_combined.gif', all_frames, duration=0.1)  # Save as a single gif with 0.1 second delay between frames
print("Finished combining all animated GIFs into one.")